In [5]:
import yfinance as yf
import pandas as pd
from datetime import datetime
import numpy as np

In [6]:
symbol_list_ibrx = list(pd.read_csv('ibrx.csv',delimiter=";").index)
symbol_list_ibrx

['RRRP3',
 'ALSO3',
 'ALPA4',
 'ABEV3',
 'ARZZ3',
 'ASAI3',
 'AURE3',
 'AZUL4',
 'B3SA3',
 'BPAN4',
 'BBSE3',
 'BBDC3',
 'BBDC4',
 'BRAP4',
 'BBAS3',
 'BRKM5',
 'BRFS3',
 'BPAC11',
 'CRFB3',
 'CBAV3',
 'CCRO3',
 'CMIG4',
 'CIEL3',
 'COGN3',
 'CPLE6',
 'CSAN3',
 'CPFE3',
 'CMIN3',
 'CVCB3',
 'CYRE3',
 'DXCO3',
 'ECOR3',
 'ELET3',
 'ELET6',
 'EMBR3',
 'ENAT3',
 'ENBR3',
 'ENGI11',
 'ENEV3',
 'EGIE3',
 'EQTL3',
 'EZTC3',
 'FLRY3',
 'GGBR4',
 'GOAU4',
 'GOLL4',
 'GMAT3',
 'NTCO3',
 'SOMA3',
 'HAPV3',
 'HYPE3',
 'IGTI11',
 'ITSA4',
 'ITUB4',
 'JBSS3',
 'KLBN11',
 'RENT3',
 'LWSA3',
 'LREN3',
 'MDIA3',
 'MGLU3',
 'MRFG3',
 'CASH3',
 'BEEF3',
 'MOVI3',
 'MRVE3',
 'MULT3',
 'PCAR3',
 'PETR3',
 'PETR4',
 'RECV3',
 'PRIO3',
 'PETZ3',
 'PSSA3',
 'QUAL3',
 'RADL3',
 'RAIZ4',
 'RDOR3',
 'RAIL3',
 'SBSP3',
 'SANB11',
 'STBP3',
 'SMTO3',
 'CSNA3',
 'SLCE3',
 'SUZB3',
 'TAEE11',
 'VIVT3',
 'TIMS3',
 'TOTS3',
 'TRPL4',
 'UGPA3',
 'USIM5',
 'VALE3',
 'VAMO3',
 'VIIA3',
 'VBBR3',
 'WEGE3',
 'YDUQ3']

In [3]:
list_melhores = []
list_relacao_mm = []
list_precos = []

soma_fundamentos = 0
dividend_yield = 0
chamada_api = yf.Ticker('PETR4.SA').history(period='30mo',interval = '1wk')
hist_precos = chamada_api['Close']
hist_dividendos = chamada_api['Dividends']
hist_dividendos_ultimo_ano = hist_dividendos.loc[datetime(2020,1,1):datetime(2021,12,31)]
    

dividend_yield = (hist_dividendos_ultimo_ano.sum()/2)/hist_precos[-1] if hist_precos[-1] > 0 else 0
if dividend_yield > 0.03:

    balanco_anual = yf.Ticker('PETR4.SA').balance_sheet
    balanco_trimestral = yf.Ticker('PETR4.SA').quarterly_balance_sheet
    dre_anual = yf.Ticker('PETR4.SA').financials
    dfc_anual = yf.Ticker('PETR4.SA').cashflow
    dre_trimestral = yf.Ticker('PETR4.SA').quarterly_financials
    dfc_trimestral = yf.Ticker('PETR4.SA').quarterly_cashflow
    
    if dre_anual.keys()[0].year >= 0:
        balanco = balanco_anual
        dre = dre_anual
        dfc = dfc_anual
        variacao_receita = int(dre.loc['Total Revenue'][0] > dre.loc['Total Revenue'][1])
        variacao_ebit = int(dre.loc['Ebit'][0] > dre.loc['Ebit'][1])
        variacao_caixa = int(dfc.loc['Change In Cash'][0] > dfc.loc['Change In Cash'][1])
        variacao_divida = int(balanco.loc['Total Current Liabilities'][0] < balanco.loc['Total Current Liabilities'][1])
        
        soma_fundamentos = variacao_caixa + variacao_ebit + variacao_receita + variacao_divida
        if soma_fundamentos >= 2:
            print("ativo",variacao_divida,variacao_caixa,variacao_ebit,variacao_receita,dre_anual.keys()[0])
            list_relacao_mm.append(abs(hist_precos[-1] / np.mean(hist_precos[-40:])-1))
            list_precos.append(round(hist_precos[-1],2))


C:\Users\igorb\AppData\Local\Temp\ipykernel_21220\1732305370.py:10: FutureWarning: Indexing a timezone-aware DatetimeIndex with a timezone-naive datetime is deprecated and will raise KeyError in a future version. Use a timezone-aware object instead.
  hist_dividendos_ultimo_ano = hist_dividendos.loc[datetime(2020,1,1):datetime(2021,12,31)]


Exception: yfinance failed to decrypt Yahoo data response

In [4]:
balanco_anual = yf.Ticker('TSLA')
balanco_anual.option_chain()

Options(calls=          contractSymbol             lastTradeDate  strike  lastPrice     bid  \
0    TSLA230331C00020000 2023-03-21 14:16:29+00:00    20.0     170.45  170.15   
1    TSLA230331C00025000 2023-03-14 16:11:23+00:00    25.0     156.85  165.15   
2    TSLA230331C00030000 2023-03-24 17:14:05+00:00    30.0     159.00  160.25   
3    TSLA230331C00035000 2023-03-22 19:08:55+00:00    35.0     159.25  155.25   
4    TSLA230331C00040000 2023-03-01 15:22:45+00:00    40.0     160.65  150.25   
..                   ...                       ...     ...        ...     ...   
96   TSLA230331C00320000 2023-03-24 14:52:43+00:00   320.0       0.01    0.00   
97   TSLA230331C00330000 2023-03-22 18:20:15+00:00   330.0       0.01    0.00   
98   TSLA230331C00340000 2023-03-17 19:54:38+00:00   340.0       0.01    0.00   
99   TSLA230331C00350000 2023-03-21 14:41:15+00:00   350.0       0.01    0.00   
100  TSLA230331C00360000 2023-03-24 15:20:54+00:00   360.0       0.01    0.00   

        ask  

In [7]:
list_melhores = []
list_relacao_mm = []
list_precos = []
for ativo in symbol_list_ibrx:
    soma_fundamentos = 0
    dividend_yield = 0
    chamada_api = yf.Ticker(ativo+'.SA')
    hist = chamada_api.history(period='30mo',interval = '1wk')
    hist_precos = hist['Close']
    hist_dividendos = hist['Dividends']
    hist_dividendos_ultimo_ano = hist_dividendos.loc[datetime(2020,1,1):datetime(2021,12,31)]
    

    dividend_yield = (hist_dividendos_ultimo_ano.sum()/2)/hist_precos[-1] if hist_precos[-1] > 0 else 0
    
    if dividend_yield > 0.03:
        try:
            balanco_anual = chamada_api.balance_sheet
            balanco_trimestral = chamada_api.quarterly_balance_sheet
            dre_anual = chamada_api.financials
            dfc_anual = chamada_api.cashflow
            dre_trimestral = chamada_api.quarterly_financials
            dfc_trimestral = chamada_api.quarterly_cashflow
            
            if dre_anual.keys()[0].year >= 0:
                balanco = balanco_anual
                dre = dre_anual
                dfc = dfc_anual
                variacao_receita = int(dre.loc['Total Revenue'][0] > dre.loc['Total Revenue'][1])
                variacao_ebit = int(dre.loc['Ebit'][0] > dre.loc['Ebit'][1])
                variacao_caixa = int(dfc.loc['Change In Cash'][0] > dfc.loc['Change In Cash'][1])
                variacao_divida = int(balanco.loc['Total Current Liabilities'][0] < balanco.loc['Total Current Liabilities'][1])
                
                soma_fundamentos = variacao_caixa + variacao_ebit + variacao_receita + variacao_divida
                if soma_fundamentos >= 2:
                    print(ativo,variacao_divida,variacao_caixa,variacao_ebit,variacao_receita,dre_anual.keys()[0])
                    list_melhores.append(ativo)
                    list_relacao_mm.append(abs(hist_precos[-1] / np.mean(hist_precos[-40:])-1))
                    list_precos.append(round(hist_precos[-1],2))
            else:
                continue
        except:
            print("erro")
            continue

C:\Users\igorb\AppData\Local\Temp\ipykernel_20168\1154706494.py:11: FutureWarning: Indexing a timezone-aware DatetimeIndex with a timezone-naive datetime is deprecated and will raise KeyError in a future version. Use a timezone-aware object instead.
  hist_dividendos_ultimo_ano = hist_dividendos.loc[datetime(2020,1,1):datetime(2021,12,31)]
C:\Users\igorb\AppData\Local\Temp\ipykernel_20168\1154706494.py:11: FutureWarning: Indexing a timezone-aware DatetimeIndex with a timezone-naive datetime is deprecated and will raise KeyError in a future version. Use a timezone-aware object instead.
  hist_dividendos_ultimo_ano = hist_dividendos.loc[datetime(2020,1,1):datetime(2021,12,31)]
C:\Users\igorb\AppData\Local\Temp\ipykernel_20168\1154706494.py:11: FutureWarning: Indexing a timezone-aware DatetimeIndex with a timezone-naive datetime is deprecated and will raise KeyError in a future version. Use a timezone-aware object instead.
  hist_dividendos_ultimo_ano = hist_dividendos.loc[datetime(2020,1

erro


C:\Users\igorb\AppData\Local\Temp\ipykernel_20168\1154706494.py:11: FutureWarning: Indexing a timezone-aware DatetimeIndex with a timezone-naive datetime is deprecated and will raise KeyError in a future version. Use a timezone-aware object instead.
  hist_dividendos_ultimo_ano = hist_dividendos.loc[datetime(2020,1,1):datetime(2021,12,31)]
C:\Users\igorb\AppData\Local\Temp\ipykernel_20168\1154706494.py:11: FutureWarning: Indexing a timezone-aware DatetimeIndex with a timezone-naive datetime is deprecated and will raise KeyError in a future version. Use a timezone-aware object instead.
  hist_dividendos_ultimo_ano = hist_dividendos.loc[datetime(2020,1,1):datetime(2021,12,31)]
C:\Users\igorb\AppData\Local\Temp\ipykernel_20168\1154706494.py:11: FutureWarning: Indexing a timezone-aware DatetimeIndex with a timezone-naive datetime is deprecated and will raise KeyError in a future version. Use a timezone-aware object instead.
  hist_dividendos_ultimo_ano = hist_dividendos.loc[datetime(2020,1

erro


C:\Users\igorb\AppData\Local\Temp\ipykernel_20168\1154706494.py:11: FutureWarning: Indexing a timezone-aware DatetimeIndex with a timezone-naive datetime is deprecated and will raise KeyError in a future version. Use a timezone-aware object instead.
  hist_dividendos_ultimo_ano = hist_dividendos.loc[datetime(2020,1,1):datetime(2021,12,31)]


...

erro


C:\Users\igorb\AppData\Local\Temp\ipykernel_20168\1154706494.py:11: FutureWarning: Indexing a timezone-aware DatetimeIndex with a timezone-naive datetime is deprecated and will raise KeyError in a future version. Use a timezone-aware object instead.
  hist_dividendos_ultimo_ano = hist_dividendos.loc[datetime(2020,1,1):datetime(2021,12,31)]
C:\Users\igorb\AppData\Local\Temp\ipykernel_20168\1154706494.py:11: FutureWarning: Indexing a timezone-aware DatetimeIndex with a timezone-naive datetime is deprecated and will raise KeyError in a future version. Use a timezone-aware object instead.
  hist_dividendos_ultimo_ano = hist_dividendos.loc[datetime(2020,1,1):datetime(2021,12,31)]
C:\Users\igorb\AppData\Local\Temp\ipykernel_20168\1154706494.py:11: FutureWarning: Indexing a timezone-aware DatetimeIndex with a timezone-naive datetime is deprecated and will raise KeyError in a future version. Use a timezone-aware object instead.
  hist_dividendos_ultimo_ano = hist_dividendos.loc[datetime(2020,1

erro


C:\Users\igorb\AppData\Local\Temp\ipykernel_20168\1154706494.py:11: FutureWarning: Indexing a timezone-aware DatetimeIndex with a timezone-naive datetime is deprecated and will raise KeyError in a future version. Use a timezone-aware object instead.
  hist_dividendos_ultimo_ano = hist_dividendos.loc[datetime(2020,1,1):datetime(2021,12,31)]
C:\Users\igorb\AppData\Local\Temp\ipykernel_20168\1154706494.py:11: FutureWarning: Indexing a timezone-aware DatetimeIndex with a timezone-naive datetime is deprecated and will raise KeyError in a future version. Use a timezone-aware object instead.
  hist_dividendos_ultimo_ano = hist_dividendos.loc[datetime(2020,1,1):datetime(2021,12,31)]


erro


In [8]:
list_precos

[]

In [1]:
carteira_df = pd.DataFrame({'MM':list_relacao_mm,'Preços':list_precos}, index=list_melhores).sort_values(by = 'MM',ascending=True)

carteira_df

NameError: name 'pd' is not defined

In [2]:
list_pesos = [(1+x)/(carteira_df['MM'].sum()+len(carteira_df['MM'].values)) for x in carteira_df['MM'].values]
list_pesos.reverse()

list_pesos

NameError: name 'carteira_df' is not defined

In [3]:
carteira_df['Pesos'] = list_pesos
carteira_df

NameError: name 'list_pesos' is not defined